In [1]:
# load libraries
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
from PIL import Image

C:\Users\danie\anaconda3\envs\ds_master\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Smple usage

source = https://huggingface.co/arnabdhar/YOLOv8-Face-Detection

In [3]:
# download model
model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")

# load model
model = YOLO(model_path)

# inference
image_path = "sample/2022-11-30_16-58-45_UTC_6.jpg"
image = Image.open(image_path)

# Perform inference
output = model(image)
              
results = Detections.from_ultralytics(output[0])


0: 640x512 (no detections), 203.1ms
Speed: 15.6ms preprocess, 203.1ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 512)


In [6]:
# inference
image_path2 = "sample/2014-03-21_10-14-52_UTC.jpg"
image2 = Image.open(image_path2)

# Perform inference
output2 = model(image2)
              
results2 = Detections.from_ultralytics(output2[0])


0: 640x640 1 FACE, 140.1ms
Speed: 14.6ms preprocess, 140.1ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)


In [13]:
type(results)

supervision.detection.core.Detections

In [8]:
results2

Detections(xyxy=array([[     132.13,      62.979,       210.4,      171.42]], dtype=float32), mask=None, confidence=array([    0.82998], dtype=float32), class_id=array([0]), tracker_id=None)

# Label all data

In [7]:
import os
import glob

# Replace this with the directory you want to search in
directory = '../1_download_data/data/raw/'


def get_jpg_files(folder_path):
    jpg_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.jpg'):
                jpg_files.append(os.path.join(root, file))
    return jpg_files

# Example usage
folder_path = '../1_download_data/data/raw/'
jpg_files = get_jpg_files(folder_path)
jpg_files[0:10]

['../1_download_data/data/raw/-PHbiexlr_\\2015-11-18_18-26-59_UTC.jpg',
 '../1_download_data/data/raw/3YO1o0Rlkt\\2015-06-01_07-45-08_UTC.jpg',
 '../1_download_data/data/raw/4yqgDyxloK\\2015-07-06_10-38-30_UTC.jpg',
 '../1_download_data/data/raw/B-y2Fo8qcN_\\2020-04-10_08-57-20_UTC.jpg',
 '../1_download_data/data/raw/B3__d73i46x\\2019-10-24_11-49-40_UTC.jpg',
 '../1_download_data/data/raw/B8zBgZeD8bQ\\2020-02-20_17-34-19_UTC.jpg',
 '../1_download_data/data/raw/B9UltWvIDjA\\2020-03-04_18-25-36_UTC.jpg',
 '../1_download_data/data/raw/B9XY-IjjDWt\\2020-03-05_20-32-01_UTC_1.jpg',
 '../1_download_data/data/raw/B9XY-IjjDWt\\2020-03-05_20-32-01_UTC_2.jpg',
 '../1_download_data/data/raw/BBIHpekRlhD\\2016-01-29_14-48-24_UTC.jpg']

In [ ]:
from PIL import Image, ImageFile
from tqdm import tqdm

ImageFile.LOAD_TRUNCATED_IMAGES = True  # Allow loading of truncated images
dict_face_detection = {}
for image in tqdm(jpg_files):
    short_name = image.split('\\')[1]
    try:
        with Image.open(image) as img:
            raw_image = img.convert('RGB')
            # Further processing of raw_image if needed
            # unconditional image captioning
            text = "a picture of "
            inputs = processor(raw_image, text, return_tensors="pt").to(device)
            out = model.generate(**inputs, num_beams = 3)
            dict_captions[short_name] = processor.decode(out[0], skip_special_tokens=True)
    except IOError as e:
        print(f"Cannot process image {image}: {e}")
        dict_captions[short_name] = ''